# Generate docs from taxo

## Read the XBRL-instance

In [1]:
from arelle import ModelManager, Cntlr, ModelFormulaObject, ModelXbrl, ViewFileFormulae, XbrlConst
from arelle import RenderingEvaluator

In [2]:
import src
import pandas as pd
from os import listdir, walk, makedirs, environ
from os.path import isfile, join, exists, basename
from io import StringIO

In [3]:
# make sure you have a 'arelle' directory in the data_path! (This is where the taxonomy is stored)
XBRL_DATA_PATH    = 'C:\\Users\\wjwil\\20_local_data\\xbrl\\' 
#DATA_PATH    = 'H:\\20_local_data\\xbrl\\' 
XBRL_RESULTS_PATH = 'C:\\Users\\wjwil\\50_results\\xbrl\\'
LANGUAGE     = "en-GB"
# set the location of taxonomy
environ['XDG_CONFIG_HOME'] = XBRL_DATA_PATH

In [4]:
DOCS_PATH ="..\\docs\\"
DATA_PATH ="..\\data\\"
RULES_PATH = "..\\Solvency2-rules\\"
DIR_NAME = "EIOPA Validation rules_2.4.0\\"

In [5]:
# Now we make a modelmanager
controller = Cntlr.Cntlr()
controller.webCache.workOffline = True

modelmanager = ModelManager.initialize(controller)
modelmanager.defaultLang = LANGUAGE
modelmanager.formulaOptions = ModelFormulaObject.FormulaOptions()
modelmanager.loadCustomTransforms()

In [15]:
template_dict = {}
datapoint_dict = {}
instances = [file for file in listdir(XBRL_DATA_PATH) if file.endswith(".xbrl")]
# And we read the XBRL instance
for file in instances:
    print(file)
    xbrl_instance = ModelXbrl.load(modelmanager, join(XBRL_DATA_PATH, file))
    linkRoleUris = xbrl_instance.relationshipSet("Table-rendering").linkRoleUris
    for role_uri in linkRoleUris:
        definition = basename(role_uri)
        tblAxisRelSet = xbrl_instance.relationshipSet(XbrlConst.euTableAxis, role_uri)
        if len(tblAxisRelSet.modelRelationships)==0:
            tblAxisRelSet = xbrl_instance.relationshipSet((XbrlConst.tableBreakdown, XbrlConst.tableBreakdownMMDD, XbrlConst.tableBreakdown201305, XbrlConst.tableBreakdown201301, XbrlConst.tableAxis2011), role_uri)
        for rootconcept in tblAxisRelSet.rootConcepts:
            # rootconcept is a modelTable-object
            template_dict[rootconcept.definitionLabelsView[2][1]] = rootconcept.definitionLabelsView[3][1]
    tables = list(xbrl_instance.relationshipSet("Table-rendering").linkRoleUris)
    RenderingEvaluator.init(xbrl_instance)
    datapoint_dict.update(src.rc2label.rc2label_dict(xbrl_instance))

aeb_240_instance.xbrl


KeyboardInterrupt: 

## List validation rules in taxonomy

In [9]:
ViewFileFormulae.viewFormulae(xbrl_instance, XBRL_RESULTS_PATH + "formulae.csv", "header", None)
formulae = pd.read_csv(XBRL_RESULTS_PATH + "formulae.csv")
df_xbrl = formulae[formulae['Expression'].str[0:2]=="BV"]

In [10]:
len(df_xbrl.index)

1210

In [11]:
df = pd.DataFrame()
for row in df_xbrl.index:
    expr = df_xbrl.loc[row, 'Expression']
    label = df_xbrl.loc[row, 'Label']
    rule_templates, rule_datapoints, rule_id, rule_ref = src.parse_formula(expr, syntax = "XBRL")
    rule_date = "Unknown"
    df = df.append(pd.DataFrame(data = [[rule_id, label, rule_templates, rule_datapoints, rule_ref, expr]]), ignore_index = True)
df.columns = ['Rule id', 'Rule label', 'Rule templates', 'Rule datapoints', 'Rule references', 'Rule expression']

In [12]:
df

,Rule id,Rule label,Rule templates,Rule datapoints,Rule references,Rule expression
0,BV925-1,s2md_BV925-1,"[S.28.02.01.01, S.28.02.01.03, S.28.02.01.06]","[S.28.02.01.01,r0010,c0010, S.28.02.01.03,r020...",None,"BV925-1: {{S.28.02.01.06, r0500,c0140}} = {{S...."
1,BV925-1,s2md_BV925-1_1,"[S.28.02.01.01, S.28.02.01.03, S.28.02.01.06]","[S.28.02.01.01,r0010,c0010, S.28.02.01.03,r020...",None,"BV925-1: {{S.28.02.01.06, r0500,c0140}} = {{S...."
2,BV926-1,s2md_BV926-1,"[S.28.02.01.01, S.28.02.01.03, S.28.02.01.06]","[S.28.02.01.01,r0010,c0020, S.28.02.01.03,r020...",None,"BV926-1: {{S.28.02.01.06, r0500,c0150}} = {{S...."
3,BV926-1,s2md_BV926-1_1,"[S.28.02.01.01, S.28.02.01.03, S.28.02.01.06]","[S.28.02.01.01,r0010,c0020, S.28.02.01.03,r020...",None,"BV926-1: {{S.28.02.01.06, r0500,c0150}} = {{S...."
4,BV925-1_W,s2md_BV925-1_W_1,"[S.28.02.01.01, S.28.02.01.03, S.28.02.01.06]","[S.28.02.01.01,r0010,c0010, S.28.02.01.03,r020...",None,"BV925-1_W: {{S.28.02.01.06, r0500,c0140}} = {{..."
5,BV925-1_W,s2md_BV925-1_W,"[S.28.02.01.01, S.28.02.01.03, S.28.02.01.06]","[S.28.02.01.01,r0010,c0010, S.28.02.01.03,r020...",None,"BV925-1_W: {{S.28.02.01.06, r0500,c0140}} = {{..."
6,BV926-1_W,s2md_BV926-1_W_1,"[S.28.02.01.01, S.28.02.01.03, S.28.02.01.06]","[S.28.02.01.01,r0010,c0020, S.28.02.01.03,r020...",None,"BV926-1_W: {{S.28.02.01.06, r0500,c0150}} = {{..."
7,BV926-1_W,s2md_BV926-1_W,"[S.28.02.01.01, S.28.02.01.03, S.28.02.01.06]","[S.28.02.01.01,r0010,c0020, S.28.02.01.03,r020...",None,"BV926-1_W: {{S.28.02.01.06, r0500,c0150}} = {{..."
8,BV840-1,s2md_BV840-1_W_1,[S.17.01.02.01],"[S.17.01.02.01,r0260, S.17.01.02.01,r0280]",[(c0020-0180 )],BV840-1: [(c0020-0180 )] (not(empty({{S.17.01....
9,BV840-1,s2md_BV840-1_W,[S.17.01.02.01],"[S.17.01.02.01,r0260, S.17.01.02.01,r0280]",[(c0020-0180 )],BV840-1: [(c0020-0180 )] (not(empty({{S.17.01....


In [14]:
for row in df.index:
    l = df.loc[row, 'Rule datapoints']
    for item in l:
        if not item in d.keys():
            print(item)

S.06.02.01.01,c0170,snnn
S.08.01.01.01,c0240,snnn
S.06.02.01.01,c0170,snnn
S.08.01.01.01,c0240,snnn
S.08.01.01.01,c0240,snnn
S.08.01.01.01,c0240,snnn
S.08.01.01.01,c0240,snnn
S.08.01.01.01,c0240,snnn
S.06.02.01.01,c0170,snnn
S.06.02.01.01,c0170,snnn
S.23.01.01.01,r0030-0050
S.23.01.01.01,r0030-0050
S.23.01.01.01,r0300-0370
S.23.01.01.01,r0300-0370
S.23.01.01.01,r0300-0370
S.23.01.01.01,r0300-0370
S.23.01.01.01,r0030-0050
S.23.01.01.01,r0030-0050
S.06.02.01.01,c0170,snnn
S.06.02.01.01,c0170,snnn
S.06.02.01.01,c0170,snnn
S.06.02.01.01,c0170,snnn
S.06.02.01.01,c0170,snnn
S.06.02.01.01,c0170,snnn
S.06.02.01.01,c0170,snnn
S.06.02.01.01,c0170,snnn
S.06.02.01.01,c0170,snnn
S.06.02.01.01,c0170,snnn
S.06.02.01.01,c0170,snnn
S.06.02.01.01,c0170,snnn
S.06.02.01.01,c0170,snnn
S.06.02.01.01,c0170,snnn
S.06.02.01.01,c0170,snnn
S.06.02.01.01,c0170,snnn
S.06.02.01.01,c0170,snnn
S.06.02.01.01,c0170,snnn
S.06.02.01.01,c0170,snnn
S.06.02.01.01,c0170,snnn
S.06.02.01.01,c0170,snnn
S.06.02.01.01,c0170,snnn


## Generate subdirectories for XBRL Formulae

In [ ]:
all_templates = []
for row in df.index:
    all_templates.extend(df.loc[row, "Rule templates"])
templates = list(pd.Series(data = all_templates).sort_values().unique())

for template in templates:
    if not exists(join(DOCS_PATH, DIR_NAME, template)):
        makedirs(join(DOCS_PATH, DIR_NAME, template))

In [ ]:
src.write_rst(join(DOCS_PATH, DIR_NAME), df, template_dict, datapoint_dict)

In [ ]:
d